In [ ]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, WRITING_GRPO_CONFIG
from trl_wrapper.wrapper_config import SMOL_LM_135M

cfg = WRITING_GRPO_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
cfg.model_id_or_path = SMOL_LM_135M
cfg.max_completion_length = 32
cfg.max_prompt_length = 32
cfg.max_samples = 10

In [ ]:
wrapper = TrainerWrapper(cfg, True)

In [ ]:
wrapper.init_model()

In [ ]:
wrapper.init_data_module()

In [ ]:
wrapper.data_module.config.run_mode

In [20]:
wrapper.init_trainer()

2025-04-29 20:30:05.024 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:396 - Saving output to: ./runs/04-29-20-30-940771-smollm2-135m-instruct--antislop-only
2025-04-29 20:30:05.025 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:419 - Initializing trainer, run_name: 04-29-20-30-940771-smollm2-135m-instruct--antislop-only, wandb project: writing-grpo
2025-04-29 20:30:05.025 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:502 - Using vllm: False


In [21]:
wrapper.train()

2025-04-29 20:30:06.798 | INFO     | trl_wrapper.trainer_wrapper:train:650 - Starting training.
2025-04-29 20:30:08.986 | INFO     | model.reasoning:logger_reward:238 - ========================================
2025-04-29 20:30:08.987 | INFO     | model.reasoning:logger_reward:239 - Prompt 0: --------------------
2025-04-29 20:30:08.987 | INFO     | model.reasoning:logger_reward:240 - Write a paragraph in a casual, first-person narrative style, similar to contemporary fiction. The tone should be a blend of lightheartedness and underlying emotional vulnerability. The scene takes place shortly after the narrator has experienced a traumatic event, and is now surrounded by supportive friends, creating a somewhat surreal and comforting atmosphere.

The paragraph should begin with a reassurance from one friend, followed by light banter about the narrator's physical wounds and recovery. Introduce another friend who steers the conversation toward the narrator's emotional well-being. The narrato

Step,Training Loss
1,0.000000
2,0.000000
3,0.000000


2025-04-29 20:30:14.731 | INFO     | model.reasoning:logger_reward:238 - ========================================
2025-04-29 20:30:14.732 | INFO     | model.reasoning:logger_reward:239 - Prompt 0: --------------------
2025-04-29 20:30:14.732 | INFO     | model.reasoning:logger_reward:240 - Write a paragraph set in a gothic-tinged steampunk world. The protagonist, a resourceful woman named Aldora, is evading clockwork automatons, formerly townspeople, controlled by a distant enemy. Maintain a tone of suspense and restrained anxiety. The paragraph should begin with Aldora observing these automatons and recalling a specific, now-altered individual – the former baker. Describe her conflicted feelings – the past familiarity juxtaposed with the present danger. Hint at a complex relationship with her father, marked by both protectiveness and a certain level of detachment, suggesting her perilous life was anticipated. Briefly introduce a male companion, John, and his unrequited feelings for Al

KeyboardInterrupt: 

In [ ]:
import pandas as pd
dataset_pd = pd.read_parquet(
    "../dataset_files/gutenberg_backtranslate_from_txt.parquet"
)
completion, instruction = dataset_pd.loc[500]['text'], dataset_pd.loc[500]['instruction']

In [ ]:
max_score = 0
for i in range(10000):
    completion = dataset_pd.loc[i]['text']
    score = wrapper.data_module.bench.calculate_slop_index(completion)
    if score > max_score:
        max_score = score
        print(score)
print(max_score)

In [ ]:
wrapper.data_module.reward_functions()[0]([instruction], [completion])